<a href="https://colab.research.google.com/github/kesavan7287/SocGenVBA/blob/main/SocGenVBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf cohere pinecone-client langchain langchain_pinecone langchain_cohere -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:

# Set environment variables
os.environ["COHERE_API_KEY"] = "gij6DlDMOwcvn1KTlrahs34BbXiUJ88TbrF1vwOn"
os.environ['PINECONE_API_KEY'] = '63cc481a-df02-4a58-8dea-cc4905005f70'
os.environ['PINECONE_ENV'] = 'gcp-starter'

# Initialize Cohere client
cohere_client = cohere.Client(api_key=os.environ["COHERE_API_KEY"])

# Initialize Cohere embeddings
embeddings = CohereEmbeddings(model="embed-english-v3.0")

# Example documents (realistic text snippets)
docs = column_y


# Initialize PineconeVectorStore with documents for retrieval
index_name = "sg1"
PineconeVectorStore.from_texts(
    texts=docs,
    embedding=embeddings,  # Use Cohere embeddings
    index_name=index_name,
    metadatas = [{"description": f"{i}"} for i in column_x]
)

# Define your chat prompt template
system_template = "System message template: {context}"
human_template = "Human message template: {question}"

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template)
]

chat_prompt = ChatPromptTemplate.from_messages(messages)

# Use the Cohere client directly as the LLM
llm = Cohere(cohere_api_key=os.environ["COHERE_API_KEY"])

# Create your ConversationalRetrievalChain with Cohere as LLM
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=PineconeVectorStore.from_existing_index(index_name,embeddings).as_retriever(),
    combine_docs_chain_kwargs={"prompt": chat_prompt},
    chain_type="stuff"  # Add chain type for Cohere
)

# Example usage
query = "what is your name"
response = qa_chain.run({"question": query,"chat_history":""})

print("Response:", response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response:  My name is Coral. I am a sophisticated chatbot that is designed to assist users by providing fact-based responses in polite, helpful, and inclusive conversations. Is there anything else you'd like to ask or discuss? 


In [ ]:
general_system_template = r"""
You are here to help people understand what VBA Macros do. Read through the entire macro and give the user the gist of what
it does. Do not explain the code line by line unless asked to do so. Keep your answer within 5 bullet points or 75 words
{context}
"""
general_user_template = "Question:```{question}```"
messages = [
            SystemMessagePromptTemplate.from_template(general_system_template),
            HumanMessagePromptTemplate.from_template(general_user_template)
]
qa_prompt = ChatPromptTemplate.from_messages( messages )


# Use the Cohere client directly as the LLM
llm = Cohere(cohere_api_key=os.environ["COHERE_API_KEY"])

# Create your ConversationalRetrievalChain with Cohere as LLM
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=PineconeVectorStore.from_existing_index(index_name,embeddings).as_retriever(),
    combine_docs_chain_kwargs={"prompt": qa_prompt},
    chain_type="stuff"  # Add chain type for Cohere
)

response = qa_chain.run({"question": query,"chat_history":""})

print("Response:", response)

Response:  • This VBA macro, named Macro1, creates a new table on the active sheet and applies a specific table style to it.
• It sets the alignment, indentation, and text wrapping options for the table's cells.
• It removes the diagonal borders and sets the line style for the other border lines of the table's cells.
• Finally, it selects all cells and fits the entire column width to accommodate its content. 

The macro is assigned a keyboard shortcut of Ctrl+r for easier access. 


In [ ]:

response = qa_chain.run({"question": query,"chat_history":""})

print("Response:", response)

Response:  The given code is for Microsoft Excel and written in Excel's Macro-Enabled Excel 4 Macro Language (XLM). 

Let's go through the code step by step:
1. **Sub Macro1()** — This line defines the start of a macro named Macro1. macros in Excel are routines that can be executed to automate different tasks.
2. **'Macro1 Macro** — This is simply a comment in the code stating that the following code is for the Macro1 macro. These comments are for human understanding and are not interpreted or executed as code.
3. **'Keyboard shortcut: Ctrl+r** — This is another comment indicating that the shortcut Ctrl+r can be used to trigger this macro.
4. **Dim rng As Range** — This line declares a variable named rng of type Range. The Range object in Excel VBA represents a cell or a range of cells on a worksheet.
5. **Set rng = ActiveSheet.Range(“A1”).CurrentRegion** — Here, the rng variable is set to refer to the current region of the range A1 (i.e., the entire row). CurrentRegion automatically s

In [ ]:
!pip install -qU langchain-cohere oletools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 989.4/989.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
import os
import cohere
from langchain_pinecone import PineconeVectorStore
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_cohere import CohereEmbeddings
# from langchain.llms import Cohere  # Import Cohere from langchain.llms
from langchain_cohere.llms import Cohere
from langchain_core.messages import SystemMessage

In [ ]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# PDF reader
import pymupdf

# Open some document, for example a PDF (could also be EPUB, XPS, etc.)
doc = pymupdf.open("/content/VBA_Cheatsheet_Zero_To_Mastery_V1.01.pdf")

# Load a desired page. This works via 0-based numbers
page = doc[11]  # this is the first page

# Look for tables on this page and display the table count
tabs = page.find_tables()
print(f"{len(tabs.tables)} table(s) on {page}")
tab = tabs[1]
column_x = []
column_y = []
c = 0
for line in tab.extract():
  if c==0:
    c+=1
    continue

  column_x.append(line[0])
  column_y.append(line[-3])

print(column_x)
print(column_y)

2 table(s) on page 11 of /content/VBA_Cheatsheet_Zero_To_Mastery_V1.01.pdf
['Target a single cell using a hard-coded\nreference', 'Target multiple cells using a hard-coded\nreference', 'Print the value of a cell using a hard-\ncoded reference', 'Set the value of a cell using a hard-\ncoded reference', 'Print the value of the active cell', 'Set the value of the active cell', 'Print the value of the cell 1 row below,\nand 2 columns to the right, of the active\ncell']
['Range(“A1”)', 'Range(“A1:C3”)', 'MsgBox(Range(“A1”).Value)', 'Range(“A1”).Value = 11', 'MsgBox(ActiveCell.Value)', 'ActiveCell.Value = 22', 'MsgBox(ActiveCell.Offset(1,2))']


In [ ]:
os.environ["COHERE_API_KEY"] = "gij6DlDMOwcvn1KTlrahs34BbXiUJ88TbrF1vwOn"
os.environ['PINECONE_API_KEY'] = '63cc481a-df02-4a58-8dea-cc4905005f70'
os.environ['PINECONE_ENV'] = 'gcp-starter'

# Initialize Cohere client
cohere_client = cohere.Client(api_key=os.environ["COHERE_API_KEY"])

# Initialize Cohere embeddings
embeddings = CohereEmbeddings(model="embed-english-v3.0")

# Example documents (realistic text snippets)
docs = column_y


# Initialize PineconeVectorStore with documents for retrieval
index_name = "sg1"
PineconeVectorStore.from_texts(
    texts=docs,
    embedding=embeddings,  # Use Cohere embeddings
    index_name=index_name,
    metadatas = [{"description": f"{i}"} for i in column_x]
)


In [ ]:
# prompt: get a vba code from xlsx file

from oletools.olevba import VBA_Parser, TYPE_OLE, TYPE_OpenXML, TYPE_Word2003_XML, TYPE_MHTML
import zipfile

def extract_vba(filename):
  vba_parser = VBA_Parser(filename)
  if vba_parser.detect_vba_macros():
    for (filename, stream_path, vba_filename, vba_code) in vba_parser.extract_macros():
      query=vba_code
      break
  else:
    print('No VBA macros found.')

# Replace 'your_excel_file.xlsm' with the actual path to your Excel file
extract_vba('/content/Excel_Macros_Chart_XelPlus.xlsm')


In [ ]:
general_system_template = r"""
You are here to help people understand what VBA Macros do. Read through the entire macro and give the user the gist of what
it does. Do not explain the code line by line unless asked to do so. Keep your answer within 5 bullet points or 75 words
{context}
"""
general_user_template = "Question:```{question}```"
messages = [
            SystemMessagePromptTemplate.from_template(general_system_template),
            HumanMessagePromptTemplate.from_template(general_user_template)
]
qa_prompt = ChatPromptTemplate.from_messages( messages )


# Use the Cohere client directly as the LLM
llm = Cohere(cohere_api_key=os.environ["COHERE_API_KEY"])
retriever=PineconeVectorStore.from_existing_index(embedding=embeddings, index_name='sg1').as_retriever()
# Create your ConversationalRetrievalChain with Cohere as LLM


### Contextualize question ###
contextualize_q_system_prompt = """You are here to help people understand what VBA Macros do. \
Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is.
{question}"""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """You are here to help people understand what VBA Macros do. Read through the entire macro and \
give the user the gist of what it does. Do not explain the code line by line unless asked to do so. \
Keep your answer within 75 words\
Do not try to run the code \
If the user asks about a specific macro by name, give the latest record about the macro \
unless they ask for previous versions \
Return the answer in the following format if you can: \
Macro Name: 'name of macro' \
Macro Use: 'formatting, aggregating, etc.' \
Macro Description: 'Description' \
Make sure the what you return is formatted correctly. \
Do not give newline characters in your responses \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
    output_parser=StrOutputParser(),
)
# response = conversational_rag_chain.invoke({"question": query, "context": "formatting tables"})
response = conversational_rag_chain.invoke(
    {"input": query, "context": "formatting tables"},
    config={"configurable": {"session_id": "foo"}}
)

response['answer']

" Macro Name: Macro1\n\nMacro Use: Applies a table style to a selected range and modifies the formatting of the selection\n\nMacro Description: 'Applies a specific table style to the selected range and modifies the formatting of the selection. Formatting modifications include centering the text horizontally, aligning the text to the bottom vertically, turning off text wrapping, adjusting orientation, indentations, merging cells, and applying thin borders to the left, top, bottom, and right edges. Additionally, the code selects all cells and fits the entire column to the screen.' "

In [ ]:
response['answer']

" Macro Name: Macro1\n\nMacro Use: Applies a table style to a selected range and modifies the formatting of the selection\n\nMacro Description: The 'Macro1' macro creates a table with the style 'TableStyleMedium27', and aligns the text to the center, bottom of the selected range. It also sets formatting options such as text wrap, orientation, indent, shrink to fit, reading order, merge cells, and border style for the selection. Finally, it auto-fits the entire column of the selected range. "

In [ ]:
import re
matc = re.search(r"Macro Name:\s*(.*?)\s+Macro Use:\s*(.*?)\s+Macro Description:\s*(.*)", response['answer'])
if matc:
    macro_name = matc.group(1).strip()
    macro_use = matc.group(2).strip()
    macro_description = matc.group(3).strip()
    print("Macro Name:", macro_name)
    print("Macro Use:", macro_use)
    print("Macro Description:", macro_description)
else:
    print("Macro Name and Description not found in the response.", response['answer'])

index_name = "logs"
docs = [f"{macro_name} {macro_description}"]
query = [{"vba_code": query}]  # List of dictionaries with VBA code as metada

Macro Name: Macro1
Macro Use: Applies a table style to a selected range and modifies the formatting of the selection
Macro Description: 'Applies a specific table style to the selected range and modifies the formatting of the selection. Formatting modifications include centering the text horizontally, aligning the text to the bottom vertically, turning off text wrapping, adjusting orientation, indentations, merging cells, and applying thin borders to the left, top, bottom, and right edges. Additionally, the code selects all cells and fits the entire column to the screen.'


In [ ]:
PineconeVectorStore.from_texts(
    texts=docs,
    embedding=embeddings,  # Use Cohere embeddings
    index_name="logs",
    metadatas = query
)

In [ ]:
system_template = '''You are in charge of logs of VBA macros\
return the code along with its name and description \
in the following format: \
Macro Name: 'name of macro' \
Macro Use: 'formatting, aggregating, etc.' \
Macro Description: 'Description' \
{context}'''
human_template = "Return the latest code: {question}"

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template)
]

chat_prompt = ChatPromptTemplate.from_messages(messages)

# Use the Cohere client directly as the LLM
llm = Cohere(cohere_api_key=os.environ["COHERE_API_KEY"])

# Create your ConversationalRetrievalChain with Cohere as LLM
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=PineconeVectorStore.from_existing_index(embedding=embeddings, index_name='logs').as_retriever(),
    combine_docs_chain_kwargs={"prompt": chat_prompt},
    chain_type="stuff"  # Add chain type for Cohere
)

# Example usage
query = "give macro for formatting tables"
response = qa_chain.run({"question": query,"chat_history":""})

print("Response:", response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response:  Macro Name: Macro1
Macro Use: Formatting
Macro Description: The macro applies specific table style and formatting options to a selected range on an Excel spreadsheet. It also creates a table using the Excel table feature and applies the "TableStyleMedium27" style to it.

Sub Macro1()
    ' Apply table style and formatting options to the selected range
    ActiveSheet.RangeSelection.FormatConditions.AddTableStyleRange("TableStyleMedium27")
    ActiveSheet.RangeSelection.FormatCells MergeAndCenter:=True
    ActiveSheet.RangeSelection.VerticalAlignment = xlBottom
    ActiveSheet.RangeSelection.WrapText = False
    ActiveSheet.RangeSelection.Orientation = xlHorizontal
    ActiveSheet.RangeSelection.IndentationMode = xlIndentDefault
    ActiveSheet.RangeSelection.MergingType = xlMergeCellsAll
    ActiveSheet.RangeSelection.Borders(xlEdgeLeft).LineStyle = xlThin
    ActiveSheet.RangeSelection.Borders(xlEdgeTop).LineStyle = xlThin
    ActiveSheet.RangeSelection.Borders(xlEdgeBottom